# JupyterHub Firefox Launcher Integration Test

This notebook tests the JupyterLab Firefox launcher extension through JupyterHub to ensure:
1. Static files are accessible through JupyterHub proxy
2. Authentication bypass works correctly
3. Compression handling is functional
4. iframe embedding support is working
5. No 405 or 500 errors occur

In [1]:
import requests
import json
import time
import os
from IPython.display import display, HTML, IFrame

# Configuration for testing
JUPYTERHUB_URL = "http://localhost:8000"
JUPYTERLAB_URL = "http://localhost:8890"
USERNAME = "bdx"  # Your username

print("🧪 JupyterHub Firefox Launcher Integration Test")
print("=" * 55)
print(f"JupyterHub URL: {JUPYTERHUB_URL}")
print(f"JupyterLab URL: {JUPYTERLAB_URL}")
print(f"Test User: {USERNAME}")

🧪 JupyterHub Firefox Launcher Integration Test
JupyterHub URL: http://localhost:8000
JupyterLab URL: http://localhost:8890
Test User: bdx


## Test 1: Direct JupyterLab Access

First, let's verify that the Firefox launcher extension works correctly when accessing JupyterLab directly.

In [2]:
def test_static_files_direct():
    """Test static file access through direct JupyterLab."""
    print("🔧 Testing direct JupyterLab static file access...")
    
    base_url = f"{JUPYTERLAB_URL}/firefox-launcher"
    test_files = [
        "js/lib/jquery.js",
        "js/lib/rencode.js",
        "js/Client.js",
        "css/client.css"
    ]
    
    results = []
    for file_path in test_files:
        url = f"{base_url}/{file_path}"
        try:
            # Test both HEAD and GET
            head_resp = requests.head(url, timeout=5)
            get_resp = requests.get(url, timeout=5)
            
            success = head_resp.status_code == 200 and get_resp.status_code == 200
            results.append({
                "file": file_path,
                "head_status": head_resp.status_code,
                "get_status": get_resp.status_code,
                "size": len(get_resp.content),
                "success": success
            })
            
            status = "✅" if success else "❌"
            print(f"   {status} {file_path}: HEAD {head_resp.status_code}, GET {get_resp.status_code}, {len(get_resp.content)} bytes")
            
        except Exception as e:
            results.append({"file": file_path, "error": str(e), "success": False})
            print(f"   ❌ {file_path}: {e}")
    
    successful = sum(1 for r in results if r["success"])
    print(f"\n📊 Direct access result: {successful}/{len(test_files)} files accessible")
    return results

direct_results = test_static_files_direct()

🔧 Testing direct JupyterLab static file access...
   ✅ js/lib/jquery.js: HEAD 200, GET 200, 289782 bytes
   ✅ js/lib/rencode.js: HEAD 200, GET 200, 14061 bytes
   ✅ js/Client.js: HEAD 200, GET 200, 157464 bytes
   ✅ css/client.css: HEAD 200, GET 200, 9411 bytes

📊 Direct access result: 4/4 files accessible


## Test 2: JupyterHub Proxy Access

Now let's test accessing the same static files through the JupyterHub proxy to ensure proper routing.

In [3]:
def test_static_files_jupyterhub():
    """Test static file access through JupyterHub proxy."""
    print("🔧 Testing JupyterHub proxy static file access...")
    
    # JupyterHub proxy path format: /user/{username}/firefox-launcher/
    base_url = f"{JUPYTERHUB_URL}/user/{USERNAME}/firefox-launcher"
    test_files = [
        "js/lib/jquery.js",
        "js/lib/rencode.js",
        "js/Client.js",
        "css/client.css"
    ]
    
    results = []
    for file_path in test_files:
        url = f"{base_url}/{file_path}"
        try:
            # Test GET request (HEAD might not work through proxy)
            get_resp = requests.get(url, timeout=10, allow_redirects=True)
            
            success = get_resp.status_code == 200
            results.append({
                "file": file_path,
                "status": get_resp.status_code,
                "size": len(get_resp.content) if success else 0,
                "success": success
            })
            
            status = "✅" if success else "❌"
            if success:
                print(f"   {status} {file_path}: {get_resp.status_code}, {len(get_resp.content)} bytes")
            else:
                print(f"   {status} {file_path}: {get_resp.status_code}")
            
        except Exception as e:
            results.append({"file": file_path, "error": str(e), "success": False})
            print(f"   ❌ {file_path}: {e}")
    
    successful = sum(1 for r in results if r["success"])
    print(f"\n📊 JupyterHub proxy result: {successful}/{len(test_files)} files accessible")
    return results

# Note: This might fail initially if JupyterHub hasn't spawned the user server yet
print("⚠️  Note: If this fails, it might be because JupyterHub hasn't spawned the user server yet.")
print("   Try accessing JupyterHub in browser first to spawn the server.")
jupyterhub_results = test_static_files_jupyterhub()

⚠️  Note: If this fails, it might be because JupyterHub hasn't spawned the user server yet.
   Try accessing JupyterHub in browser first to spawn the server.
🔧 Testing JupyterHub proxy static file access...
   ✅ js/lib/jquery.js: 200, 289782 bytes
   ✅ js/lib/rencode.js: 200, 14061 bytes
   ✅ js/Client.js: 200, 157464 bytes
   ✅ css/client.css: 200, 9411 bytes

📊 JupyterHub proxy result: 4/4 files accessible


## Test 3: Client Endpoint Authentication Bypass

Test that the client endpoint works without authentication (crucial for iframe embedding).

In [4]:
def test_client_endpoint_bypass():
    """Test client endpoint authentication bypass."""
    print("🔐 Testing client endpoint authentication bypass...")
    
    # Test direct JupyterLab access
    direct_url = f"{JUPYTERLAB_URL}/firefox-launcher/client?ws=ws://localhost:12345/&http=http://localhost:12345/"
    
    try:
        response = requests.get(direct_url, timeout=5)
        direct_success = response.status_code == 200
        print(f"   Direct access: {response.status_code} {'✅' if direct_success else '❌'}")
        
        if direct_success:
            # Check if it returns HTML content
            is_html = 'text/html' in response.headers.get('content-type', '')
            print(f"   Content type: {response.headers.get('content-type', 'unknown')} {'✅' if is_html else '❌'}")
        
        return {
            "direct_status": response.status_code,
            "direct_success": direct_success,
            "content_type": response.headers.get('content-type', 'unknown')
        }
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return {"error": str(e), "direct_success": False}

client_test = test_client_endpoint_bypass()

🔐 Testing client endpoint authentication bypass...
   Direct access: 200 ✅
   Content type: text/html; charset=utf-8 ✅


## Test 4: Compression Handling

Test that various compression formats are handled correctly (this was the original zlib issue).

In [5]:
def test_compression_handling():
    """Test compression handling for different encodings."""
    print("🗜️  Testing compression handling...")
    
    base_url = f"{JUPYTERLAB_URL}/firefox-launcher/js/Client.js"
    
    compression_tests = [
        {"name": "No compression", "headers": {}},
        {"name": "Gzip", "headers": {"Accept-Encoding": "gzip"}},
        {"name": "Deflate", "headers": {"Accept-Encoding": "deflate"}},
        {"name": "Brotli", "headers": {"Accept-Encoding": "br"}},
        {"name": "Multiple", "headers": {"Accept-Encoding": "gzip, deflate, br"}}
    ]
    
    results = []
    for test in compression_tests:
        try:
            response = requests.get(base_url, headers=test["headers"], timeout=5)
            success = response.status_code == 200 and len(response.content) > 0
            
            results.append({
                "name": test["name"],
                "status": response.status_code,
                "size": len(response.content),
                "encoding": response.headers.get("content-encoding", "none"),
                "success": success
            })
            
            status = "✅" if success else "❌"
            print(f"   {status} {test['name']}: {response.status_code}, {len(response.content)} bytes, encoding: {response.headers.get('content-encoding', 'none')}")
            
        except Exception as e:
            results.append({"name": test["name"], "error": str(e), "success": False})
            print(f"   ❌ {test['name']}: {e}")
    
    successful = sum(1 for r in results if r["success"])
    print(f"\n📊 Compression test result: {successful}/{len(compression_tests)} encodings working")
    return results

compression_results = test_compression_handling()

🗜️  Testing compression handling...
   ✅ No compression: 200, 157464 bytes, encoding: none
   ✅ Gzip: 200, 157464 bytes, encoding: none
   ✅ Deflate: 200, 157464 bytes, encoding: none
   ✅ Brotli: 200, 157464 bytes, encoding: none
   ✅ Multiple: 200, 157464 bytes, encoding: none

📊 Compression test result: 5/5 encodings working


## Test 5: iframe Embedding Test

Create an actual iframe to test that the extension can be embedded properly.

In [6]:
def test_iframe_embedding():
    """Test iframe embedding capabilities."""
    print("🖼️  Testing iframe embedding...")
    
    # Test CSP headers
    test_url = f"{JUPYTERLAB_URL}/firefox-launcher/css/client.css"
    
    try:
        response = requests.get(test_url, timeout=5)
        csp_header = response.headers.get("Content-Security-Policy", "")
        
        # Check if frame-ancestors allows embedding
        iframe_friendly = (
            "frame-ancestors *" in csp_header or
            "frame-ancestors 'self'" in csp_header or
            "frame-ancestors" not in csp_header
        )
        
        print(f"   CSP Header: {csp_header or 'None'}")
        print(f"   iframe friendly: {'✅' if iframe_friendly else '❌'}")
        
        # Create a simple iframe test
        iframe_url = f"{JUPYTERLAB_URL}/firefox-launcher/client?ws=ws://localhost:12345/&http=http://localhost:12345/"
        
        iframe_html = f"""
        <div style="border: 2px solid #007acc; padding: 10px; margin: 10px 0;">
            <h4>🖼️ iframe Embedding Test</h4>
            <p>If the iframe loads without errors, embedding is working:</p>
            <iframe 
                src="{iframe_url}" 
                width="100%" 
                height="400px" 
                style="border: 1px solid #ccc;"
                sandbox="allow-scripts allow-same-origin allow-forms">
                Your browser does not support iframes.
            </iframe>
            <p><small>URL: {iframe_url}</small></p>
        </div>
        """
        
        display(HTML(iframe_html))
        
        return {
            "csp_header": csp_header,
            "iframe_friendly": iframe_friendly,
            "test_url": iframe_url
        }
        
    except Exception as e:
        print(f"   ❌ Error testing iframe: {e}")
        return {"error": str(e)}

iframe_test = test_iframe_embedding()

🖼️  Testing iframe embedding...
   CSP Header: frame-ancestors 'self'; report-uri /api/security/csp-report
   iframe friendly: ✅


## Test Summary and Results

Comprehensive summary of all test results and recommendations.

In [ ]:
def print_test_summary():
    """Print comprehensive test summary."""
    print("🎯 JupyterHub Firefox Launcher Test Summary")
    print("=" * 50)
    
    # Count successes
    direct_success = sum(1 for r in direct_results if r.get("success", False))
    jupyterhub_success = sum(1 for r in jupyterhub_results if r.get("success", False))
    compression_success = sum(1 for r in compression_results if r.get("success", False))
    
    print(f"\n📊 Test Results:")
    print(f"   Direct JupyterLab access: {direct_success}/4 static files ({'✅' if direct_success >= 3 else '❌'})")
    print(f"   JupyterHub proxy access: {jupyterhub_success}/4 static files ({'✅' if jupyterhub_success >= 3 else '❌'})")
    print(f"   Client endpoint bypass: {'✅' if client_test.get('direct_success', False) else '❌'}")
    print(f"   Compression handling: {compression_success}/5 encodings ({'✅' if compression_success >= 4 else '❌'})")
    print(f"   iframe embedding: {'✅' if iframe_test.get('iframe_friendly', False) else '❌'}")
    
    # Overall assessment
    overall_score = (
        (direct_success >= 3) +
        (jupyterhub_success >= 3) +
        client_test.get('direct_success', False) +
        (compression_success >= 4) +
        iframe_test.get('iframe_friendly', False)
    )
    
    print(f"\n🏆 Overall Score: {overall_score}/5")
    
    if overall_score >= 4:
        print("🎉 EXCELLENT: Firefox launcher is working great through JupyterHub!")
        print("   • All critical issues resolved")
        print("   • Ready for production use")
        print("   • iframe embedding supported")
    elif overall_score >= 3:
        print("✅ GOOD: Firefox launcher is mostly working")
        print("   • Minor issues may need attention")
        print("   • Check failed tests above")
    else:
        print("⚠️  NEEDS WORK: Some issues remain")
        print("   • Review failed tests")
        print("   • Check JupyterHub configuration")
    
    print(f"\n🔗 Quick Access URLs:")
    print(f"   JupyterHub: {JUPYTERHUB_URL}")
    print(f"   JupyterLab: {JUPYTERLAB_URL}")
    print(f"   Firefox Launcher: {JUPYTERLAB_URL}/firefox-launcher/")

print_test_summary()

🎯 JupyterHub Firefox Launcher Test Summary

📊 Test Results:
   Direct JupyterLab access: 4/4 static files (✅)
   JupyterHub proxy access: 4/4 static files (✅)
   Client endpoint bypass: ✅
   Compression handling: 5/5 encodings (✅)
   iframe embedding: ✅

🏆 Overall Score: 5/5
🎉 EXCELLENT: Firefox launcher is working great through JupyterHub!
   • All critical issues resolved
   • Ready for production use
   • iframe embedding supported

🔗 Quick Access URLs:
   JupyterHub: http://localhost:8000
   JupyterLab: http://localhost:8890
   Firefox Launcher: http://localhost:8890/firefox-launcher/


: 